# Preliminaries

In [2]:
# import modules
import numpy as np
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# base classes

class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

# Density Tree

In [4]:
class DensityTree(Tree):
    def __init__(self):
        super(DensityTree, self).__init__()
        
    def train(self, data, prior, n_min=20):
        '''
        data: the feature matrix for the digit under consideration
        prior: the prior probability of this digit
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        self.prior = prior
        N, D = data.shape
        D_try = int(np.sqrt(D)) # number of features to consider for each split decision

        # find and remember the tree's bounding box, 
        # i.e. the lower and upper limits of the training feature set
        m, M = np.min(data, axis=0), np.max(data, axis=0)
        self.box = m.copy(), M.copy()
        
        # identify invalid features and adjust the bounding box
        # (If m[j] == M[j] for some j, the bounding box has zero volume, 
        #  causing divide-by-zero errors later on. We must exclude these
        #  features from splitting and adjust the bounding box limits 
        #  such that invalid features have no effect on the volume.)
        valid_features   = np.where(m != M)[0]
        invalid_features = np.where(m == M)[0]
        M[invalid_features] = m[invalid_features] + 1

        # initialize the root node
        self.root.data = data
        self.root.box = m.copy(), M.copy()

        # build the tree
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min:
                # Call 'make_density_split_node()' with 'D_try' randomly selected 
                # indices from 'valid_features'. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                rand_indices = np.random.permutation(valid_features)[:D_try]
                left, right = make_density_split_node(node, N, rand_indices)
                stack.append(left)
                stack.append(right)
            else:
                # Call 'make_density_leaf_node()' to turn 'node' into a leaf node.
                make_density_leaf_node(node, N)

    def predict(self, x):
        leaf = self.find_leaf(x)
        # return p(x | y) * p(y) if x is within the tree's bounding box 
        # and return 0 otherwise   
        m, M = self.box
        if np.any(x < m) or np.any(x > M):
            return 0
        else:
            return self.prior * leaf.response

In the density tree section. First, we tried to find and remember the tree's bounding box. Then we tried to identify invalid features
and adjust the bounding box. While building the tree we tried to call 'make_density_split_node()' with D_try. This turned node into a split node
and returned the two children. 

In [5]:
def make_density_split_node(node, N, feature_indices):
    '''
    node: the node to be split
    N:    the total number of training instances for the current class
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape
    m, M = node.box

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    j_min, t_min = 0, 0
    v = np.prod(M-m)
    
    for j in feature_indices:
        # Hint: For each feature considered, first remove duplicate feature values using 
        # 'np.unique()'. Describe here why this is necessary.
        data_unique = np.sort(np.unique((node.data[:, j])))
        # Compute candidate thresholds
        tj = (data_unique[:-1] + data_unique[1:]) / 2
        
        # Illustration: for loop - hint: vectorized version is possible
        for t in tj:
          N_l = np.sum(t >= node.data[:,j]) 
          N_r = n - N_l

          V_l = v * t / (M[j] - m[j])
          V_r = v - V_l

          error_l = N_l / (N * V_l) * (N_l / N - 2 * (N_l-1) / (N-1))
          error_r = N_r / (N * V_r) * (N_r / N - 2 * (N_r-1) / (N-1))
          # Compute the error
          loo_error = error_l + error_r
            
            # choose the best threshold that
          if loo_error < e_min:
              e_min = loo_error
              j_min = j
              t_min = t

    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and bounding boxes
    # according to the optimal split found above
    left.data = node.data[node.data[:,j_min] <= t_min, :] # store data in left node -- for subsequent splits
    left.box = m.copy(), M.copy() # store bounding box in left node
    left.box[1][j_min] = t_min
    right.data = node.data[node.data[:,j_min] > t_min, :]
    right.box = m.copy(), M.copy()
    right.box[0][j_min] = t_min

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min
    node.threshold = t_min

    # return the children (to be placed on the stack)
    return left, right

In the decision tree, we tried to minimize the Gini impurity of the resulting children. 

In [6]:
def make_density_leaf_node(node, N):
    '''
    node: the node to become a leaf
    N:    the total number of training instances for the current class
    '''
    # compute and store leaf response
    n = node.data.shape[0]
    v = np.prod(node.box[1] - node.box[0])
    node.response = n / (N * v)

# Decision Tree

In [7]:
class DecisionTree(Tree):
    def __init__(self):
        super(DecisionTree, self).__init__()
        
    def train(self, data, labels, n_min=20):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D)) # how many features to consider for each split decision

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min and not node_is_pure(node):
                # Call 'make_decision_split_node()' with 'D_try' randomly selected 
                # feature indices. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                ... # your code here
                res_perm = np.random.permutation(D)   # permute D indices
                left, right = make_decision_split_node(node, res_perm[:D_try])

                stack.append(left)
                stack.append(right)
            else:
                # Call 'make_decision_leaf_node()' to turn 'node' into a leaf node.
                # your code here
                make_decision_leaf_node(node)
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # compute p(y | x)
        return leaf.response  # your code here

In [14]:
def make_decision_split_node(node, feature_indices):
    '''
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape

    # find best feature j (among 'feature_indices') and best threshold t for the split
    # your code here
    googol = 1e100
    j_min, t_min = 0, 0
    for j in feature_indices:
        # remove duplicate features
        data_unique = np.sort(np.unique(node.data[:,j]))
        # compute candidate thresholds in the middle between consecutive feature values
        tj = (data_unique[:-1] + data_unique[1:]) / 2
        # each candidate threshold we need to compute Gini impurities of the resulting children node
        for t in tj:
            N_l = np.sum(t >= node.data[:,j]) 
            N_r = n - N_l
            lk_l = node.labels[t >= node.data[:,j]]
            error_l = N_l * (1 - np.sum(np.square(np.bincount(lk_l)/N_l)))
            lk_r = node.labels[node.data[:,j] > t]
            error_r = N_r * (1 - np.sum(np.square(np.bincount(lk_r)/N_r)))
            # choose the the best threshold that minimizes sum of Gini impurities
            if googol > error_l + error_r:
                googol = error_l + error_r
                j_min = j
                t_min = t
    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and labels
    # according to the optimal split found above
    left.data = node.data[node.data[:,j_min] <= t_min, :]
    left.labels = node.labels[node.data[:,j_min] <= t_min]
    right.data = node.data[node.data[:,j_min] > t_min, :]
    right.labels = node.labels[node.data[:,j_min] > t_min]

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min
    node.threshold = t_min

    # return the children (to be placed on the stack)
    return left, right    

In the function make_decision_split_node we tried to find the best feature j and best threhold t for the split. 
We tried to remove duplicate features so that in the end we won't have more than one best feature. 

In [9]:
def make_decision_leaf_node(node):
    '''
    node: the node to become a leaf
    '''
    # compute and store leaf response
    node.N = node.labels.shape[0]
    node.response = np.bincount(node.labels, minlength=10) / node.N

In [10]:
def node_is_pure(node):
    '''
    check if 'node' cntains only instances of the same digit
    '''
    return np.unique(node.labels).shape[0] == 1

# Evaluation of Density and Decision Tree

In [11]:
digits = load_digits()
data = digits["data"]
target = digits["target"]
data_subsets = [data[target==i] for i in range(10)]

In [12]:
class GenClassifier:
    def __init__(self):
        #Create 10 Instances For Density Tree 
        self.trees = [DensityTree() for i in range(10)]
    
    def train(self, data, target, n_min=20):
        #Train 10 Instances
        data_subsets = [data[target==i] for i in  range(10)] #each with one subset of digits
        N = len(target)
        for i, tree in enumerate(self.trees):
            tree.train(data_subsets[i], len(data_subsets[i]) / N, n_min)
            
    def predict(self, x):
        #Return the Digits for the Density Tree
        return np.argmax([tree.predict(x) for tree in self.trees])

In the function GenClassifier we tried to use one instance of DensityTree per digit class and a discriminative classifier using one instance of DecisionTree. We tried to use different n_min to improve the performance.  

What we did here is: we created 10 instances of Density Tree. We trained 10 trees, each with one subset of digits. And returned the digits for the Density Tree that maximized p(x | y)*p(y)

In [27]:
n_min = 20


# train with full dataset
# training generative classifier with DensityTrees
GenC = GenClassifier()
GenC.train(data, target, n_min)

decTree = DecisionTree()
decTree.train(data, target, n_min)

confusion_GenC = np.zeros((10,10))
confusion_DenC = np.zeros((10,10))
# for each digit subset
for i in range(10):
    # predict for with generative classifier
    predictions = np.array([GenC.predict(j) for j in data_subsets[i]])
    confusion_GenC[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100
    # predict for with discriminative tive classifier
    predictions = np.array([np.argmax(decTree.predict(j)) for j in data_subsets[i]])
    confusion_DenC[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100
print('Generative Classifier using 10 instances of Density Tree')
display(pd.DataFrame(data = confusion_GenC, index =range(10), columns =range(10)))
print('Discriminative Classifier using 1 instance of Decision Tree')
display(pd.DataFrame(data = confusion_DenC, index =range(10), columns =range(10)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in true_divide


Generative Classifier using 10 instances of Density Tree


,0,1,2,3,4,5,6,7,8,9
0,100.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,69.230769,0.549451,0.000000,0.000000,0.549451,0.000000,0.000000,23.626374,6.043956
2,0.0,10.169492,57.062147,15.254237,0.000000,2.824859,0.000000,0.000000,14.689266,0.000000
3,0.0,2.732240,0.000000,65.573770,0.000000,4.918033,0.000000,1.639344,22.404372,2.732240
4,0.0,0.552486,0.000000,0.000000,87.845304,0.552486,0.000000,9.944751,1.104972,0.000000
5,0.0,0.549451,0.000000,1.098901,0.549451,77.472527,0.000000,2.747253,11.538462,6.043956
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,99.447514,0.000000,0.552486,0.000000
7,0.0,0.000000,0.000000,0.558659,3.351955,1.675978,0.000000,94.413408,0.000000,0.000000
8,0.0,7.471264,0.574713,1.149425,0.000000,1.149425,0.000000,0.000000,89.655172,0.000000
9,0.0,0.555556,0.000000,8.333333,1.666667,1.111111,0.000000,6.111111,12.777778,69.444444


Discriminative Classifier using 1 instance of Decision Tree


,0,1,2,3,4,5,6,7,8,9
0,92.696629,0.000000,0.000000,0.000000,0.000000,0.561798,3.932584,0.000000,0.561798,2.247191
1,0.000000,83.516484,6.043956,0.000000,0.549451,0.549451,1.098901,0.549451,7.692308,0.000000
2,0.564972,5.649718,87.005650,2.259887,0.564972,0.000000,0.564972,0.000000,2.824859,0.564972
3,0.546448,3.278689,0.000000,89.617486,0.000000,0.000000,0.000000,0.546448,4.918033,1.092896
4,0.552486,2.762431,0.000000,0.000000,90.607735,1.104972,1.104972,2.762431,0.000000,1.104972
5,0.549451,3.846154,0.549451,2.197802,0.000000,81.868132,3.296703,1.648352,1.098901,4.945055
6,0.552486,0.552486,0.552486,0.000000,0.000000,2.209945,91.160221,0.000000,3.867403,1.104972
7,1.117318,1.675978,0.000000,0.000000,0.000000,0.558659,0.000000,94.972067,0.558659,1.117318
8,0.000000,4.022989,0.574713,3.448276,2.298851,0.574713,0.574713,0.000000,83.333333,5.172414
9,1.666667,5.000000,0.000000,2.777778,1.111111,0.000000,0.000000,3.333333,1.111111,85.000000


Here, we trained GenerativeClassifier with one DecsionTree. We used n_min = 20

In [29]:
n_min = 5


# train with full dataset
# training generative classifier with DensityTrees
GenC = GenClassifier()
GenC.train(data, target, n_min)

decTree = DecisionTree()
decTree.train(data, target, n_min)

confusion_GenC = np.zeros((10,10))
confusion_DenC = np.zeros((10,10))
# for each digit subset
for i in range(10):
    # predict for with generative classifier
    predictions = np.array([GenC.predict(j) for j in data_subsets[i]])
    confusion_GenC[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100
    # predict for with discriminative tive classifier
    predictions = np.array([np.argmax(decTree.predict(j)) for j in data_subsets[i]])
    confusion_DenC[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100
print('Generative Classifier using 10 instances of Density Tree')
display(pd.DataFrame(data = confusion_GenC, index =range(10), columns =range(10)))
print('Discriminative Classifier using 1 instance of Decision Tree')
display(pd.DataFrame(data = confusion_DenC, index =range(10), columns =range(10)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':


Generative Classifier using 10 instances of Density Tree


,0,1,2,3,4,5,6,7,8,9
0,99.438202,0.000000,0.000000,0.000000,0.561798,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,72.527473,9.340659,0.000000,0.000000,0.549451,0.000000,2.197802,12.637363,2.747253
2,0.000000,2.259887,59.887006,3.389831,0.000000,2.259887,0.000000,0.000000,31.638418,0.564972
3,0.000000,1.639344,1.092896,55.191257,0.000000,3.825137,0.000000,3.278689,25.136612,9.836066
4,0.000000,3.314917,0.000000,0.000000,84.530387,0.000000,0.000000,10.497238,1.104972,0.552486
5,0.000000,1.098901,0.000000,0.000000,0.549451,83.516484,0.000000,4.945055,7.142857,2.747253
6,0.000000,0.000000,0.552486,0.000000,0.000000,0.000000,98.342541,0.000000,1.104972,0.000000
7,0.000000,0.558659,0.000000,0.000000,1.117318,1.117318,0.000000,96.648045,0.558659,0.000000
8,0.000000,4.022989,0.574713,0.000000,0.574713,1.149425,0.000000,5.747126,87.931034,0.000000
9,0.000000,3.888889,0.000000,3.333333,1.666667,3.333333,0.000000,6.111111,5.555556,76.111111


Discriminative Classifier using 1 instance of Decision Tree


,0,1,2,3,4,5,6,7,8,9
0,99.438202,0.000000,0.000000,0.000000,0.000000,0.561798,0.000000,0.000000,0.000000,0.000000
1,0.000000,98.901099,0.000000,0.549451,0.000000,0.549451,0.000000,0.000000,0.000000,0.000000
2,0.000000,2.259887,95.480226,1.129944,0.000000,0.000000,0.000000,1.129944,0.000000,0.000000
3,0.000000,0.546448,0.546448,96.174863,0.000000,0.546448,0.000000,1.639344,0.000000,0.546448
4,0.000000,1.657459,1.104972,0.000000,96.132597,0.552486,0.000000,0.000000,0.552486,0.000000
5,0.000000,0.549451,0.549451,1.648352,1.098901,96.153846,0.000000,0.000000,0.000000,0.000000
6,1.104972,0.552486,0.000000,0.552486,0.000000,0.552486,97.237569,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.558659,0.000000,1.117318,0.558659,0.000000,97.765363,0.000000,0.000000
8,0.000000,2.298851,1.149425,0.000000,1.149425,0.574713,0.000000,0.574713,93.678161,0.574713
9,0.000000,1.666667,0.555556,1.111111,1.111111,0.555556,0.000000,0.000000,1.111111,93.888889


Here, we trained GenerativeClassifier with one DecsionTree. We used n_min = 5

In [30]:
n_min = 10


# train with full dataset
# training generative classifier with DensityTrees
GenC = GenClassifier()
GenC.train(data, target, n_min)

decTree = DecisionTree()
decTree.train(data, target, n_min)

confusion_GenC = np.zeros((10,10))
confusion_DenC = np.zeros((10,10))
# for each digit subset
for i in range(10):
    # predict for with generative classifier
    predictions = np.array([GenC.predict(j) for j in data_subsets[i]])
    confusion_GenC[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100
    # predict for with discriminative tive classifier
    predictions = np.array([np.argmax(decTree.predict(j)) for j in data_subsets[i]])
    confusion_DenC[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100
print('Generative Classifier using 10 instances of Density Tree')
display(pd.DataFrame(data = confusion_GenC, index =range(10), columns =range(10)))
print('Discriminative Classifier using 1 instance of Decision Tree')
display(pd.DataFrame(data = confusion_DenC, index =range(10), columns =range(10)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in true_divide


Generative Classifier using 10 instances of Density Tree


,0,1,2,3,4,5,6,7,8,9
0,100.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.0,83.516484,3.296703,0.000000,0.000000,0.549451,0.0,1.648352,10.439560,0.549451
2,0.0,6.779661,68.926554,3.389831,0.000000,0.000000,0.0,0.000000,20.338983,0.564972
3,0.0,1.092896,7.103825,59.016393,0.000000,3.825137,0.0,2.732240,24.043716,2.185792
4,0.0,2.762431,0.000000,0.000000,85.635359,0.552486,0.0,6.629834,1.104972,3.314917
5,0.0,1.648352,0.000000,7.142857,0.000000,78.571429,0.0,3.846154,7.142857,1.648352
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,100.0,0.000000,0.000000,0.000000
7,0.0,3.351955,0.000000,0.558659,0.000000,1.117318,0.0,92.737430,1.117318,1.117318
8,0.0,15.517241,2.298851,0.000000,0.000000,1.149425,0.0,2.298851,78.735632,0.000000
9,0.0,4.444444,1.111111,16.111111,1.111111,2.777778,0.0,2.777778,8.333333,63.333333


Discriminative Classifier using 1 instance of Decision Tree


,0,1,2,3,4,5,6,7,8,9
0,96.629213,0.000000,0.000000,0.561798,0.561798,0.000000,0.000000,0.000000,0.000000,2.247191
1,0.549451,92.307692,0.549451,0.549451,0.000000,2.197802,1.098901,1.098901,1.648352,0.000000
2,1.129944,4.519774,92.655367,0.564972,0.000000,0.000000,0.000000,0.000000,1.129944,0.000000
3,0.000000,1.092896,3.278689,93.989071,0.000000,0.000000,0.000000,0.000000,0.546448,1.092896
4,1.657459,1.657459,0.552486,0.000000,93.922652,0.000000,0.552486,0.552486,0.552486,0.552486
5,1.648352,0.000000,0.000000,0.549451,1.098901,93.956044,0.549451,0.000000,1.098901,1.098901
6,1.104972,0.000000,1.104972,0.000000,1.657459,1.104972,93.922652,0.000000,0.552486,0.552486
7,0.000000,0.000000,0.000000,1.117318,1.675978,0.000000,0.000000,96.648045,0.558659,0.000000
8,1.724138,2.298851,2.298851,2.873563,0.000000,3.448276,0.574713,1.149425,83.908046,1.724138
9,0.555556,0.555556,0.555556,1.666667,0.555556,1.666667,0.000000,2.222222,1.111111,91.111111


As a result, in our first case we noticed that increasing n_min decreases the training quality. And when we tried to decrease n_min increased the training quality for both classifiers.
..

# Density and Decision Forest

In [ ]:
class DensityForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DensityTree() for i in range(n_trees)]
    
    def train(self, data, prior, n_min=20):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

In [ ]:
class DecisionForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DecisionTree() for i in range(n_trees)]
    
    def train(self, data, labels, n_min=0):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

# Evaluation of Density and Decision Forest

In [ ]:
# train forests (with 20 trees per forest), plot training error confusion matrices, and comment on your results
... # your code here